In [23]:
import os
import sys
import glob
import datetime
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import xray
import random
import itertools
import urllib
import json
import importlib as imp

from IPython import display

sys.path.append('/nuwa_cluster/home/jackyu/climatools/')
import climatools.dates as climadates
import climatools.viz as climaviz
import climatools.misc as climamisc
import climatools.muths as climaths
import climatools.units as climaunits
import climatools.geocoords as climageocoords
import climatools.prepdatasets as climaprepdata

sys.path.append('/nuwa_cluster/home/jackyu/radiation/rtmtools/')
import rtmtools.rrtmg.sw.info as rrtmgswinfo

#from IPython.lib.deepreload import reload as dreload
imp.reload(climadates)
imp.reload(climaviz)
imp.reload(climamisc)
imp.reload(climaths)
imp.reload(climaunits)
imp.reload(climaprepdata)

imp.reload(rrtmgswinfo)


matplotlib.rcParams.update({'font.size': 14, 
                            'xtick.direction': 'out',
                            'ytick.direction': 'out'})

# ARM95

2 cases considered here are:

* a no aerosol --- LW RRTMG, SW RRTMG, output sent to CAM
* b aerosol in SW --- LW RRTMG, SW RRTMG, aerosol in SW, output sent to CAM



In [37]:
iopname = 'arm95_LW_RRTMG_SW_RRTMG_vs_LW_RRTMG_SW_RRTMGaer'

def case_names():
    'Give the different cases names'
    return ['a no aerosol', 'b aerosol in SW']



def get_cases_difference():
    '''
    define the differences between cases to compute
    '''
    return [('b aerosol in SW', 'a no aerosol'),]



def get_cases():
    '''
    Returns a dictionary where the keys are cases, such
    as a name for a SCAM run, and the values are the datasets (xray)
    for the cases
    '''
    ds_a = xray.open_dataset('../examples/ARM95/cam5_noavg_LW_RRTMG_SW_RRTMG/camrun.cam.h0.0095-07-18-19800.nc',
                                decode_times = False)
    ds_b = xray.open_dataset('../examples/ARM95/cam5_noavg_LW_RRTMG_SW_RRTMGaer/camrun.cam.h0.0095-07-18-19800.nc',
                                  decode_times = False)    
    return dict(zip(case_names(), [ds_a, ds_b]))




def compute_net_solar_fluxes(datasets):
    '''
    Compute net solar fluxes
    '''
    for casename, ds in datasets.items():
        ds['net_FS'] = ds['FDS'] - ds['FUS']
        ds['net_FS'].attrs['units'] = ds['FDS'].attrs['units']
        ds['net_FS'].attrs['long_name'] = 'net solar flux'
        ds['net_FS'].attrs['case_name'] = casename
        
        ds['net_FSC'] = ds['FDSC'] - ds['FUSC']
        ds['net_FSC'].attrs['units'] = ds['FDSC'].attrs['units']
        ds['net_FSC'].attrs['long_name'] = 'net solar flux (clear-sky)'
        ds['net_FSC'].attrs['case_name'] = casename
    return datasets



def compute_QR(dsets):
    '''
    Compute total heating rate
    '''
    for case, ds in dsets.items():
        ds['QR'] = (ds['QRL'].dims, ds['QRL'] + ds['QRS'], ds['QRL'].attrs)
        ds['QR'].attrs['long_name'] = 'Total heating rate'
        
        ds['QRC'] = (ds['QRLC'].dims, ds['QRLC'] + ds['QRSC'], ds['QRLC'].attrs)
        ds['QRC'].attrs['long_name'] = 'Total clearsky heating rate'
    return dsets



def linestyles_by_case():
    '''
    Assign lineplot properties to the cases
    '''
    return dict(zip(case_names(), 
                    [{'colour': 'g', 'linestyle': '--'},
                     {'colour': 'darkorange', 'linestyle': '-.'}]))



def get_d3interests():
    '''
    List here all variables of interest that have dimensions (time, lev, lon, lat)
    '''
    d3interests = ['CLOUD', 'QRL', 'QRS', 'QR', 'QRLC', 'QRSC', 'QRC',
                   'Q', 
                   'CLDICE', 'CLDLIQ', 
                   'AQSNOW', 'AREI', 'AREL',
                   'AWNC', 'AWNI', 
                   'DCQ', 'DTCOND', 
                   'ICIMR', 'ICWMR', 
                   'NUMLIQ', 'NUMICE', 'T', 'RELHUM',
                   'FUL', 'FDL', 'FULC', 'FDLC', 'net_FS', 'net_FSC',
                   'FUS', 'FDS', 'FUSC', 'FDSC',
                   'TOT_CLD_VISTAU',]
    return sorted(d3interests)




def get_d2interests():
    '''
    List here all variables of interest that have dimensions (time, lon, lat)
    '''
    interests = ['FSNT', 'FSNS', 'FLNT', 'FLNS',
                 'FSNTC', 'FSNSC', 'FLNTC', 'FLNSC',
                 'CLDLOW', 'CLDMED', 'CLDHGH', 'CLDTOT',
                 'TS', 'TREFHT', 'TROP_T', 'TROP_Z', 'TMQ', 'TGCLDIWP', 'TGCLDLWP', 
                 'PRECT', 'PRECL', 'PRECC',
                 'LHFLX', 'SHFLX',
                'SOLIN',
                'AODVIS']
    return interests






def convert_units(datasets):
    '''
    Convert units for certain fields for all cases in DATASETS.
    INPUT:
    ilev --- level(interface) pressure in mbar
    lev  --- layer(level) pressure in mbar
    datetimes --- time in pandas DatetimeIndex objects
    '''
    for name, ds in datasets.items():
        
        for k in ['TOT_CLD_VISTAU',]:
            if k in ds:
                pass #ds[k].values[ds[k].isnull().values] = - 0.
        
        for k in ['FUS', 'FDS', 'net_FS', 'FUSC', 'FDSC', 'net_FSC']:
            if k in ds:
                ds[k] *= 1e3
    
        for k in ['QRL', 'QRS', 'QR', 'QRLC', 'QRSC', 'QRC', 'DTCOND']:
            if k in ds:
                ds[k] *= 86400
                ds[k].attrs['units'] = 'K/day'
        
        for k in ['Q', ]:
            if k in ds:
                ds[k] *= 1e3
                ds[k].attrs['units'] = 'g/kg'
        
        for k in ['CLDICE', 'CLDLIQ', 'AQSNOW']:
            if k in ds:
                ds[k] *= 1e6
                ds[k].attrs['units'] = 'mg/kg'
                
        for k in ['PRECC', 'PRECL', 'PRECT']:
            if k in ds:
                ds[k] *= 1e3 * 86400
                ds[k].attrs['units'] = 'mm/day'
                
    return datasets

In [25]:
def prep_data(cases):
    '''
    Preprocess data for plotting. 
    pressure and datetimes need to be provided for unit conversion.
    INPUT:
    ilev --- level(interface) pressure [mbar]
    lev --- layer(level) pressure [mbar]
    datetimes --- time in pandas DatetimeIndex objects    
    '''
    interests = get_d3interests() + get_d2interests()
    cases = climaprepdata.convert_pressure_time_units(cases)
    cases = climaprepdata.gather_interests_from_cases(cases, interests)
    cases = convert_units(cases)
    diff_cases = climaprepdata.take_difference_between_cases(cases, get_cases_difference())
    cases, diff_cases = climaprepdata.passon_attrs_casename(cases, diff_cases, interests = interests)
    return cases, diff_cases


In [26]:
################## Variables of dimensions (time, lev, lon, lat) ########################
#########################################################################################


# for writing plotted figures to disk
def writefig_contourfs(d3sets, diff_d3sets, interests, 
                       dir_label = 'example_dir_label', file_label = 'example_file_label',
                       fmt = 'png'):
    '''
    For each interest, produce one figure containing contour
    plots for all cases, another figure containing contour 
    Hackish colormap selection here, basically look up a dictionary.
    '''

    for interest in interests:
        
        dirname = 'figs_{}'.format(dir_label)
        if dirname not in glob.glob(dirname):
            os.mkdir(dirname)

            
        filename = '_'.join([interest, file_label]) if file_label else interest
            
        fig = climaviz.contourf_interest_for_all_cases(d3sets, interest,
                                           cmap = 'auto-select')
        fig.savefig(dirname + '/contourf_{}.{}'.format(filename, fmt))
        plt.close()
        
        fig = climaviz.contourf_interest_for_all_cases(diff_d3sets, interest,
                                           cmap = 'auto-select')
        fig.savefig(dirname + '/contourf_{}_diff.{}'.format(filename, fmt))
        plt.close()
        

        
def writefig_timeaveraged_VSplots(d3sets, diff_d3sets, interests, 
                                 dir_label = 'example_dir_label', file_label = 'example_file_label',
                                 fmt = 'png'):
    '''
    Average over the time dimension, plot against pressure.
    Save figure.
    '''
    for interest in interests:        
        dirname = 'figs_{}'.format(dir_label)
        if dirname not in glob.glob(dirname):
            os.mkdir(dirname)
               
        filename = '_'.join([interest, file_label]) if file_label else interest
        
        fig = climaviz.plotVS_timeaveraged_interest_for_all_cases(d3sets, diff_d3sets, 
                               interest = interest, xscale = 'linear',
                               bot_xlabels_rotate = 10., 
                               top_xlabels_rotate = 20.,
                               linestyles = linestyles_by_case(),
                               top_xlim = (- 1, 1) if interest in ['QRL', 'QRS', 'QR', 'QRLC', 'QRSC', 'QRC'] else None,
                               bot_xlim = (- 5, 1) if interest in ['QRL', 'QRLC'] \
                                                else (0, 6) if interest in ['QRS', 'QRSC'] \
                                                else (- 4, 2) if interest in ['QR', 'QRC'] else None)
        fig.savefig(dirname + '/plot_{}_timeaverage.{}'.format(filename, fmt))
        plt.close()
        

   
        
        
        
def write_d3variable_html_page(interest = 'CLOUD', 
                               dir_label = 'example_dir_label', file_label = 'example_file_label'):
    '''
    Write and save an HTML for interest.  
    Include figures produced for interest: contourfs and time-averaged line plots for example.
    HTML file named by interest and file_label, saved in a directory named by dir_label.
    '''
    figdir = 'figs_' + dir_label
    filename = '_'.join([interest, file_label]) if file_label else interest
    with open(os.path.join(figdir, 'page_{}.html'.format(filename)), 
              mode = 'w', encoding = 'utf-8') as f:
        f.write('''
                <img src="contourf_{suffix}.png" height="450">
                <img src="contourf_{suffix}_diff.png" height="450">
                <br>
                <img src="plot_{suffix}_timeaverage.png" height="500">
        '''.format(figdir = figdir, suffix = filename))

        
        
        
def write_d3variable_links_html(d3sets, interests = ['T'],
                               dir_label = 'example_dir_label', file_label = 'example_file_label'):
    '''
    Write HTML of hyperlinks to HTML files containing figures.
    A hyperlink is written for each interest. 
    '''
    hyperlinks = ['<a href="figs_{dir_label}/page_{filename}.html">{interest}: {longname}</a>'\
                 .format(dir_label = dir_label, 
                         file_label = file_label, 
                         interest = interest, 
                         filename = '_'.join([interest, file_label]) if file_label else interest,
                         longname = d3sets[random.choice(list(d3sets.keys()))][interest].attrs['long_name']) \
                 for interest in interests]
    return '<br>'.join(hyperlinks)
    

In [27]:
############### Variables of dimensions (lon, lat, time) ##########################
###################################################################################


def writefig_plotVS_timeseries(dsets, diff_dsets, interests,
                               dir_label = 'example_dir_label',
                               file_label = 'example_file_label',
                               fmt = 'png'):
    '''
    Plot and save figures of time-series of variables of interest
    '''
    for interest in interests:
        fig = climaviz.plotVS_interest_for_all_cases(dsets, diff_dsets, interest = interest,
                                                    linestyles = linestyles_by_case())
        
        dirname = 'figs_{}'.format(dir_label)
        
        if dirname not in glob.glob(dirname):
            os.mkdir(dirname)
            
        filename = '_'.join([interest, file_label]) if file_label else interest
            
        fig.savefig(dirname + '/plotVSts_{}.{}'.format(filename, fmt))
        plt.close()


        
        
def write_d2variable_html_page(interest = 'FLNS',
                               dir_label = 'example_dir_label', 
                               file_label = 'example_file_label'):
    '''
    Write and save an HTML for interest.  
    Include figures produced for interest: contourfs and time-averaged line plots for example.
    HTML file named by interest and file_label, saved in a directory named by dir_label.   
    '''
    figdir = 'figs_{}'.format(dir_label)
    filename = '_'.join([interest, file_label]) if file_label else interest
    with open(os.path.join(figdir, 'page_{}.html'\
                           .format(filename)),
             mode = 'w', encoding = 'utf-8') as f:
        f.write('''
        <img src="plotVSts_{}.png" width=700>
        '''.format(filename))
        
        
        

def write_d2variables_html(d2sets, interests = ['FLNS'],
                          dir_label = 'example_dir_label',
                          file_label = 'example_file_label'):
    hyperlinks = ['<a href="figs_{dir_label}/page_{filename}.html">{interest}: {longname}</a>'\
                 .format(dir_label = dir_label, 
                         interest = interest,
                         filename = '_'.join([interest, file_label]) if file_label else interest,
                         longname = d2sets[random.choice(list(d2sets.keys()))]\
                         [interest].attrs['long_name'])\
                 for interest in interests]
    return '<br>'.join(hyperlinks)


In [28]:
def flux_essential_facts(dsets, diff_dsets):
    casea, caseb = case_names()
    case_diff = ' - '.join(get_cases_difference()[0])
    
    d = xray.Dataset()
    
    d['insolation'] = ((), 
                       dsets[casea]['FDS'].isel(lon = 0, lat = 0).\
                       mean(dim = 'time', keep_attrs = True).isel(ilev = 0),
                      dsets[casea]['FDS'].attrs)
        
    d['TOA upward SW ({}) larger by'.format(caseb)] = ((), 
                                           diff_dsets[case_diff]['FUS'][{'lon': 0, 'lat': 0}].\
                                           mean(dim = 'time', keep_attrs = True)[{'ilev': 0}],
                                          diff_dsets[case_diff]['FUS'].attrs)
            
    d['Sfc downward SW ({}) smaller by'.format(caseb)] = ((), 
                                              - diff_dsets[case_diff]['FDS'][dict(lon = 0, lat = 0)].\
                                              mean(dim = 'time', keep_attrs = True)[{'ilev': -1}],
                                             diff_dsets[case_diff]['FDS'].attrs)
        
    d['Atmospheric SW heating ({}) smaller by'.format(caseb)] = ((),
                                                     d['TOA upward SW ({}) larger by'.format(caseb)] \
                                                               - d['Sfc downward SW ({}) smaller by'.format(caseb)],
                                                    d['TOA upward SW ({}) larger by'.format(caseb)].attrs)
    
    
    
    d['OLR ({}) smaller by'.format(caseb)] = ((), 
                                  - diff_dsets[case_diff]['FUL'][dict(lon = 0, lat = 0)].\
                                  mean(dim = 'time', keep_attrs = True).isel(ilev = 1),
                                 diff_dsets[case_diff]['FUL'].attrs)
    
    
    d['TOA (SW+LW) ({}) system cooler by'.format(caseb)] = ((), 
                                                d['TOA upward SW ({}) larger by'.format(caseb)]\
                                                          - d['OLR ({}) smaller by'.format(caseb)],
                                               d['TOA upward SW ({}) larger by'.format(caseb)].attrs)

    
    d['clear sky albedo'] = ((),
                             dsets[caseb]['FUSC'].\
                             isel(lon = 0, lat = 0).mean(dim = 'time', keep_attrs = True).\
                             isel(ilev = 0) / d['insolation'] * 100.,
                            d['insolation'].attrs)
    d['clear sky albedo'].attrs['units'] = '%'
    
    d['insolation (%)'] = ((), d['insolation'] / 1365 * 100, d['insolation'].attrs)
    d['insolation (%)'].attrs['units'] = '%'
    
    
    d['albedo TOA ({}) larger by'.format(caseb)] =  ((),
                                         d['TOA upward SW ({}) larger by'.format(caseb)] / d['insolation'] * 100.,
                                        d['insolation'].attrs)
    d['albedo TOA ({}) larger by'.format(caseb)].attrs['units'] = '%'
    
    
    name_values_units = [(k, da.values, da.attrs['units']) for k, da in d.items()]
    names, values, units = zip(*name_values_units)
    df = pd.DataFrame({'values': np.round(values, 2), 'units': units}, index = names)
    
    ordered_index = ['insolation',
                    'insolation (%)',
                    'clear sky albedo',
                    'TOA upward SW ({}) larger by'.format(caseb),
                    'albedo TOA ({}) larger by'.format(caseb),
                    'Sfc downward SW ({}) smaller by'.format(caseb),
                    'Atmospheric SW heating ({}) smaller by'.format(caseb),
                    'OLR ({}) smaller by'.format(caseb),
                    'TOA (SW+LW) ({}) system cooler by'.format(caseb)]
    return df.loc[ordered_index, ['values', 'units']]



In [29]:
def flux_atm_LW(dsets, diff_dsets, clearsky = False):
    casenames = case_names()
    diffnames = [' - '.join(diffname) for diffname in get_cases_difference()]
    location_slice = dict(lon = 0, lat = 0)
    
    name_up = 'FULC' if clearsky else 'FUL'
    name_down = 'FDLC' if clearsky else 'FDL'
    name_netup = 'FnetULC' if clearsky else 'FnetUL'
    long_name = 'Longwave {} net upward flux'.format('clearsky' if clearsky else '')
    
    for case in casenames:
        dsets[case][name_netup] = (dsets[case][name_up].dims, 
                                 dsets[case][name_up] - dsets[case][name_down],
                                dsets[case][name_up].attrs)
        dsets[case][name_netup].attrs['long_name'] = long_name
    
    for case in diffnames:
        diff_dsets[case][name_netup] = (diff_dsets[case][name_up].dims,
                                      diff_dsets[case][name_up] - diff_dsets[case][name_down],
                                     diff_dsets[case][name_up].attrs)
        diff_dsets[case][name_netup].attrs['long_name'] = long_name
        
    d_FnetUL = dict([(case, dsets[case][name_netup]) for case in casenames]\
                    + [(case, diff_dsets[case][name_netup]) for case in diffnames])
    
    d = {}
    for case, da in d_FnetUL.items():
        da_timeavg = da.isel(**location_slice).mean(dim = 'time', keep_attrs = True)
        d[case] = da_timeavg.isel(ilev = [0, -1]).values
    df = pd.DataFrame(d, index = ['Outgoing longwave radiation', 'Surface net upward'])
    df.loc['Atmosphere cooling', :] = df.loc['Outgoing longwave radiation', :] - df.loc['Surface net upward']
    df = np.round(df, decimals = 2)
    df.index.names = ['(W m-2)']
    return df



def flux_atm_SW(dsets, diff_dsets, clearsky = False):
    casenames = case_names()
    diffnames = [' - '.join(diffname) for diffname in get_cases_difference()]
    location_slice = dict(lon = 0, lat = 0)
    
    name_up = 'FUSC' if clearsky else 'FUS'
    name_down = 'FDSC' if clearsky else 'FDS'
    name_netdown = 'FnetDSC' if clearsky else 'FnetDS'
    long_name = 'Shortwave {} net downward flux'.format('clearsky' if clearsky else '')  
    
    
    for case in casenames:
        dsets[case][name_netdown] = (dsets[case][name_down].dims, 
                                 dsets[case][name_down] - dsets[case][name_up],
                                dsets[case][name_down].attrs)
        dsets[case][name_netdown].attrs['long_name'] = long_name
    
    for case in diffnames:
        diff_dsets[case][name_netdown] = (diff_dsets[case][name_down].dims,
                                      diff_dsets[case][name_down] - diff_dsets[case][name_up],
                                     diff_dsets[case][name_down].attrs)
        diff_dsets[case][name_netdown].attrs['long_name'] = long_name
        
    d_FnetUL = dict([(case, dsets[case][name_netdown]) for case in casenames]\
                    + [(case, diff_dsets[case][name_netdown]) for case in diffnames])
    
    d = {}
    for case, da in d_FnetUL.items():
        da_timeavg = da.isel(**location_slice).mean(dim = 'time', keep_attrs = True)
        d[case] = da_timeavg.isel(ilev = [0, -1]).values
    df = pd.DataFrame(d, index = ['Incoming solar radiation', 'Surface net downward'])
    df.loc['Atmosphere heating', :] = df.loc['Incoming solar radiation', :] - df.loc['Surface net downward']
    df = np.round(df, decimals = 2)
    df.index.names = ['(W m-2)']
    return df



def write_flux_atm_to_csv(dsets, diff_dsets, 
                          dir_label = 'example_dir_label', 
                          file_label = 'example_file_label'):
    
    dirname = 'data_{}'.format(dir_label)
        
    if dirname not in glob.glob(dirname):
        os.mkdir(dirname)   
        
    df_LW = flux_atm_LW(dsets, diff_dsets)
    df_SW = flux_atm_SW(dsets, diff_dsets)
    
    df_LW.to_csv(os.path.join(dirname, 'atm_flux_LW.csv'))
    df_SW.to_csv(os.path.join(dirname, 'atm_flux_SW.csv'))
    
    df_LW_clearsky = flux_atm_LW(dsets, diff_dsets, clearsky = True)
    df_SW_clearsky = flux_atm_SW(dsets, diff_dsets, clearsky = True)
    
    df_LW_clearsky.to_csv(os.path.join(dirname, 'atm_flux_LW_clearsky.csv'))
    df_SW_clearsky.to_csv(os.path.join(dirname, 'atm_flux_SW_clearsky.csv'))
    

In [30]:
cases = get_cases()
cases = compute_net_solar_fluxes(cases)
cases = compute_QR(cases)
dsets, diff_dsets = prep_data(cases) 

# write table of net flux through atmosphere to .csv files
write_flux_atm_to_csv(dsets, diff_dsets, dir_label = iopname, file_label = '')

# Variables of dimensions (time, ilev, lon, lat)

In [38]:
# plot (time, lev/ilev, lon, lat) variables, write figures to disk 

writefig_contourfs(dsets, diff_dsets, get_d3interests(), 
                   dir_label = iopname, file_label = '',
                  fmt = 'png')

writefig_timeaveraged_VSplots(dsets, diff_dsets, get_d3interests(), 
                              dir_label = iopname, file_label = '',
                              fmt = 'png')

/nuwa_cluster/home/jackyu/locally_installed_python_pacakges/anaconda3/lib/python3.4/site-packages/matplotlib/collections.py:571: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


In [39]:
# create HTML files and generate HTML page for each interest
for interest in get_d3interests():
    write_d3variable_html_page(interest = interest, dir_label = iopname, file_label = '')


display.HTML(write_d3variable_links_html(dsets, interests = sorted(get_d3interests()), dir_label = iopname, file_label = ''))

# Variables of dimensions (lon, lat, time)

In [40]:
# plot (time, lon, lat) variables, and write figures to disk and create HTML files
writefig_plotVS_timeseries(dsets, diff_dsets, get_d2interests(),
                           dir_label = iopname, file_label = '',
                           fmt = 'png')  

In [41]:
# generate HTML page for each interest
for interest in get_d2interests():
    write_d2variable_html_page(interest = interest, dir_label = iopname, file_label = '')



display.HTML(write_d2variables_html(dsets, interests = sorted(get_d2interests()),
                                   dir_label = iopname, file_label = ''))

# Flux summary

In [42]:
flux_essential_facts(dsets, diff_dsets)

,values,units
insolation,456.48,W/m2
insolation (%),33.44,%
clear sky albedo,14.19,%
TOA upward SW (b aerosol in SW) larger by,1.12,W/m2
albedo TOA (b aerosol in SW) larger by,0.24,%
Sfc downward SW (b aerosol in SW) smaller by,4.33,W/m2
Atmospheric SW heating (b aerosol in SW) smaller by,-3.21,W/m2
OLR (b aerosol in SW) smaller by,0.84,W/m2
TOA (SW+LW) (b aerosol in SW) system cooler by,0.27,W/m2


In [21]:
def slice_in_time(cases, time = slice(None)):
    return {casename: ds.sel(time = time) for casename, ds in cases.items()}

In [282]:
which_timeslice = slice('1995-07-19 05:00:00', '1995-07-25 05:00:00')

d3sets_subtime = slice_in_time(d3sets, time = which_timeslice)
diff_d3sets_subtime = slice_in_time(diff_d3sets, time = which_timeslice)

In [283]:
# plot and write figures to disk
#interests = get_d3interests()

#writefig_contourfs(d3sets_subtime, diff_d3sets_subtime, interests, 
#                   dir_label = iopname, file_label = which_timeslice,
#                  fmt = 'png')

#writefig_timeaveraged_VSplots(d3sets_subtime, diff_d3sets_subtime, interests, 
#                              dir_label = iopname, file_label = which_timeslice,
#                              fmt = 'png')


In [284]:
# generate HTML page for each interest
#for interest in get_d3interests():
#    write_d3variable_html_page(interest = interest, dir_label = iopname, file_label = which_timeslice)


#display.HTML((write_d3variable_links_html(interests = interests, dir_label = iopname, file_label = which_timeslice)))

In [43]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')